In [ ]:
%run "./Setup.ipynb"

In [ ]:
inputPath = "E:\\PySpark\\data\\users.json"

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df1 = spark.read.json(inputPath)
df1.show()

In [ ]:
df1.createOrReplaceTempView("users")

In [ ]:
spark.catalog.listTables()

In [ ]:
qry = """select age, count(1) as count
        from users
        where age is not null
        group by age
        order by count
        limit 5"""

df2 = spark.sql(qry)

df2.show()

%md
#### GlobalTempViews

In [ ]:
df1.createOrReplaceGlobalTempView("gusers")

In [ ]:
spark.catalog.listTables("global_temp")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
qry2 = """select age, count(1) as count
        from global_temp.gusers
        where age is not null
        group by age
        order by count
        limit 5"""

df3 = spark.sql(qry2)

df3.show()

#### Create a new session

In [ ]:
spark2 = spark.newSession()

In [ ]:
spark2.catalog.listTables()

In [ ]:
spark.catalog.listTables()

In [ ]:
print(qry)

In [ ]:
spark2.catalog.listTables("global_temp")

In [ ]:
spark2.sql(qry).show()

In [ ]:
print(qry2)

In [ ]:
df3 = spark2.sql(qry2)

df3.show()

#### Saving to tables

In [ ]:
df2 = df1.where("phone is not null") \
        .select("userid", "name", "gender", "age", "phone")
    
df2.show()

In [ ]:
spark.conf.get("spark.sql.warehouse.dir")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
spark.sql("create database demodb")

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.catalog.currentDatabase()

**Managed Tables**

In [ ]:
df2.write.format("json").saveAsTable("users_json")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_json where gender='Male'").show()

In [ ]:
df2.write.format("csv").option("header", True).saveAsTable("users_csv")

In [ ]:
spark.sql("select * from users_csv where age > 18").show()

**External Tables**

In [ ]:
df2.write \
    .format("csv") \
    .option("header", True) \
    .option("sep", "\t") \
    .option("path", "E:\\PySpark\\data\\external_tables\\users_external") \
    .saveAsTable("users_external")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_external limit 5").show()

**Partitioned Tables**

In [ ]:
df2.write \
    .format("json") \
    .partitionBy("gender") \
    .saveAsTable("users_partitioned")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_partitioned").show()

**drop tables and temp views**

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("drop table users_csv")

In [ ]:
spark.sql("drop table users_json")

In [ ]:
spark.sql("drop table users_partitioned")

In [ ]:
spark.sql("drop table users_external")

In [ ]:
spark.catalog.dropTempView("users")

In [ ]:
spark.catalog.listTables("global_temp")

In [ ]:
spark.catalog.dropGlobalTempView("gusers")

In [63]:
df2.rdd.getNumPartitions()

1

In [64]:
spark.conf.get("spark.sql.files.maxPartitionBytes")

'134217728b'

In [65]:
128 * 1024 * 1024

134217728